In [3]:
# Custom style
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [22]:
call_in_time_list =  [1, 2]
call_duration_list = [2, 2]
wait_times_list = []
available_reps = ["john", "cena"]
unavailable_reps = []
next_available = [0, 0]

# for time_entry in range(len(call_in_time_list)):
#     process_available_representative(available_reps, unavailable_reps, call_in_time_list[time_entry])
#     process_customer(call_in_time_list[time_entry], call_in_time_list[time_entry], wait_times_list)
#     if time_entry > (call_in_time_list[time_entry] )
#         process_unavailable_representative(available_reps, unavailable_reps)
        
        
for time_entry in range(len(call_in_time_list)):
    unavailable_reps.append(available_reps.pop(0))
    wait_times_list.append(phone_answered - call_in_time)
    nex_available[]
    
print(available_reps)
print(unavailable_reps)
print(wait_times_list)

[1]
[]
[0, 0]


In [16]:
def process_customer(call_in_time, phone_answered, wait_times_list):
    wait_times_list.append(phone_answered - call_in_time)

def process_available_representative(available_reps, unavailable_reps, time):
    unavailable_reps.append(available_reps.pop(0))
    
    
def process_unavailable_representative(available_reps, unavailable_reps):
    available_reps.append(unavailable_reps.pop(0))

# As Array

In [47]:
call_in_time_list =  [1, 2, 3]
call_duration_list = [3, 2, 2]
wait_times_list = []
next_available = [0, 0]

for time_entry in range(len(call_in_time_list)):
    end_time = call_in_time_list[time_entry] + call_duration_list[time_entry]
    next_available[next_available.index(min(next_available))] = end_time
    wait_times_list.append( end_time - call_duration_list[time_entry] - call_in_time_list[time_entry])
    
print(next_available)
print(wait_times_list)

[5, 4]
[0, 0, 0]


In [ ]:
next